In [ ]:
!pip install datasets
!pip install google-generativeai
!pip install --upgrade together

In [1]:
import re
import pandas as pd
#from datasets import load_dataset
import time
import pickle
from tqdm import tqdm

### FORMATTING DATASET TO FORMAT IN PAPER

In [ ]:
dataset = load_dataset('gsm8k', 'main')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
dataset.shape

{'train': (7473, 2), 'test': (1319, 2)}

In [ ]:
dataset['train'][0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [ ]:

def clean_answer(answer):
    # Remove content within << >> using regular expression
    cleaned_answer = re.sub(r'<<.*?>>', '', answer)
    # Replace #### with "The answer is"
    cleaned_answer = cleaned_answer.replace("####", "The answer is").replace("\n"," ")
    # Replace numbers with comma with numbers without comma
    cleaned_answer = re.sub(r'(\d{1,3}(?:,\d{3})+)', lambda x: x.group(1).replace(',', ''), cleaned_answer)
    return cleaned_answer.strip()

def clean_dataset(dataset):
    new_dataset = {'train': []}
    for data in dataset['train']:
        cleaned_answer = clean_answer(data['answer'])
        new_data = {'question': data['question'], 'answer': cleaned_answer}
        new_dataset['train'].append(new_data)
    return new_dataset

# Assuming dataset is a dictionary containing 'train' key with a list of dictionaries
t_dataset = clean_dataset(dataset)

In [ ]:
t_dataset['train'][644]

{'question': 'The city’s bus system carries 1,200,000 people each day. How many people does the bus system carry for 13 weeks?',
 'answer': '13 weeks have 13 * 7 = 91 days. For 91 days the bus system carries 1200000 * 91 = 109200000 people. The answer is 109200000'}

In [ ]:
df = pd.DataFrame(t_dataset['train'])
df['final_answer'] = df['answer'].apply(lambda x: x.split("The answer is ")[-1])
df['final_answer'] = pd.to_numeric(df['final_answer'])

In [ ]:
df.head()

,question,answer,final_answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = 24 clips in May. Natalia s...,72
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $0.2 per minute. Working 50...,10
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $50...",5
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = 24 pages today. So she was...,42
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=6 pages a week So he...,624


In [ ]:
df.columns

Index(['question', 'answer', 'final_answer'], dtype='object')

In [ ]:
df.to_pickle("/content/drive/MyDrive/NLP project/gsm8k_formatted_raw_data.pkl")

### PROMPTING

In [ ]:
df=pd.read_pickle("/content/drive/MyDrive/NLP project/gsm8k_formatted_raw_data.pkl")

In [ ]:
prompt_examples="""Example 1
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees will the grove
workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21-15=6. The answer is 6.
C: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been [21 - 15 = y1]. The answer is y1.

Example 2
Q: The flowers cost $9, the clay pot costs $20 more than the flower, and the bag of soil costs $2 less than the flower. How much does it cost to plant the flowers?
A: The clay pot costs $20 + $9 = $29. The bag of soil costs $9 - $2 = $7. The cost to plant the flowers is $9 + $29 + $7 = $45. The answer is 45.
C: The clay pot costs [20 + 9 = y1]. The bag of soil costs [9 - 2 = y2]. The cost to plant the flowers is [9 + y1 + y2 = y3]. The answer is y3.

Example 3
Q: From March to August, Sam made $460 doing 23 hours of yard work. However, from September to February, Sam was only able to work for 8 hours. If Sam
is saving up to buy a video game console that costs $600 and has already spent $340 to fix his car, how many more hours does he need to work before he can buy
the video game console?
A: Sam makes $460 / 23 hrs = $20/hr. From September to February, Sam made 8hrs x $20/hr = $160. From March to February, Sam made a total of $460 + $160
= $620. After fixing his car, he was left with $620 - $340 = $280. Sam needs another $600 - $280 = $320. Sam needs to work another $320 / $20/hr = 16 hours.
The answer is 16.
C: Sam makes [460 / 23 = y1] dollars per hour. From September to February, Sam made [8 * y1 = y2] dollars. From March to February, Sam made a
total of [460 + y2 = y3] dollars. After fixing his car, he was left with [y3 - 340 = y4]. Sam needs another [600 - y4 = y5] dollars. Sam needs to work
another [y5 / y1 = y6] hours. The answer is y6.

Example 4
Q: There were nine computers in the server room. Five more computers were installed each day, from Monday to Thursday. How many computers are now in
the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
C: There were originally 9 computers. For each of 4 days, 5 more computers were added. So [5 * 4 = y1] computers were added. [9 + y1 = y2].
The answer is y2.

Example 5
Q: Of the 90 people on William’s bus, 3/5 were Dutch. Of the 1/2 of the Dutch who were also American, 1/3 got window seats. What’s the number of Dutch
Americans who sat at the windows?
A: On the bus, the number of Dutch people was 3/5 of the total number, a total of 3/5 x 90 = 54 people. Out of the 54 people who were Dutch, 1/2 were Dutch
Americans, a total of 1/2 x 54 = 27 people. If 1/3 of the passengers on the bus identifying as Dutch Americans sat at the windows, their number is 1/3 x 27 = 9.
The answer is 9.
C: On the bus, the number of Dutch people was 3/5 of the total number, a total of [3/5 * 90 = y1] people. Out of the Dutch people, 1/2 were Dutch
Americans, a total of [1/2 * y1 = y2] people. If 1/3 of the passengers on the bus identifying as Dutch Americans sat at the windows, their number
is [1/3 * y2 = y3]. The answer is y3."""

In [ ]:
prompt_rows = df[['question', 'answer']].rename(columns={'question': 'Q', 'answer': 'A'}).to_dict('records')

#### TOGETHER AI

In [ ]:
llama_prompt=f"""Examples to generate Chain of Equations( C)  given Q and A are:

{prompt_examples}


Now  generate C for the test examples below. Please Write "###DONE" when you have completed generating C for all of the examples below.

"""

def generate_continuation_llama(prompt_rows):
    prompt = llama_prompt

    for idx, row in enumerate(prompt_rows):
        prompt += f"Test Example {idx + 1}\n"
        prompt += f"Q: {row['Q']}\n"
        prompt += f"A: {row['A']}\n"
        prompt += "C:\n\n"
    return prompt + "Respond only with C for the test examples. Use this format >> Test Example <number> C : <generated C>\nChain of Equations C for the above test examples are:"



In [ ]:
input_prompt=generate_continuation_llama(prompt_rows[0:5])

In [ ]:
print(input_prompt)

Examples to generate Chain of Equations( C)  given Q and A are:

Example 1
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees will the grove
workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21-15=6. The answer is 6.
C: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been [21 - 15 = y1]. The answer is y1.

Example 2
Q: The flowers cost $9, the clay pot costs $20 more than the flower, and the bag of soil costs $2 less than the flower. How much does it cost to plant the flowers?
A: The clay pot costs $20 + $9 = $29. The bag of soil costs $9 - $2 = $7. The cost to plant the flowers is $9 + $29 + $7 = $45. The answer is 45.
C: The clay pot costs [20 + 9 = y1]. The bag of soil costs [9 - 2 = y2]. The cost to plant the flowers is [9 + y1 + y2 = y3]. The answe

In [ ]:
import os
from together import Together

client = Together(api_key="<API KEY>")

response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    top_k= 30,
    max_tokens=2000,
    messages=[{
            "content": "You are an agent that converts math problems to a Chain of Equations. Your task to generate Chain of Equations(C) given Question (Q) and Answer (A).","role": "system"},
             {"role": "user", "content": input_prompt}],
)
print(response.choices[0].message.content)

>> Test Example 1 C : Natalia sold [48 / 2 = y1] clips in May. Natalia sold [48 + y1 = y2] clips altogether in April and May. The answer is y2.

>> Test Example 2 C : Weng earns [12 / 60 = y1] dollars per minute. Working [50 * y1 = y2] minutes, she earned [y2 = y3] dollars. The answer is y3.

>> Test Example 3 C : Betty has only [100 / 2 = y1] dollars in the beginning. Betty's parents gave her [15 * 2 = y2] dollars. This means, Betty needs [100 - y1 - y2 - 15 = y3] dollars more. The answer is y3.

>> Test Example 4 C : Julie read [12 * 2 = y1] pages today. So she was able to read a total of [12 + y1 = y2] pages since yesterday. There are [120 - y2 = y3] pages left to be read. Since she wants to read half of the remaining pages tomorrow, then she should read [y3 / 2 = y4] pages. The answer is y4.

>> Test Example 5 C : James writes each friend [3 * 2 = y1] pages a week. So he writes [y1 * 2 = y2] pages every week. That means he writes [y2 * 52 = y3] pages a year. The answer is y3.

###D

### GEMINI

In [ ]:
gemini_prompt=f"""You are an agent that converts math problems to a Chain of Equations. Examples to generate Chain of Equations( C)  given Q and A are:

### EXAMPLES
{prompt_examples}

### TASK
Now  generate C for the test examples below. Please Write "###DONE" when you have completed generating C for all of the examples below.
Respond in this format only:
Test Example <example number> C:

"""

def generate_continuation_gemini(prompt_rows):
    prompt = gemini_prompt

    for idx, row in enumerate(prompt_rows):
        prompt += f"Test Example {idx + 1}\n"
        prompt += f"Q: {row['Q']}\n"
        prompt += f"A: {row['A']}\n"
        prompt += "\n"
    return prompt + "### RESPONSE\n"


In [ ]:
import google.generativeai as genai

genai.configure(api_key="<API KEY>")
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}


model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",#"gemini-1.5-pro-latest",
                              generation_config=generation_config)

def get_muti_c(rows):
  input_prompt=generate_continuation_gemini(rows)
  prompt_parts = [input_prompt]
  response = model.generate_content(prompt_parts)
  contents = re.findall(r'C:(.*)', response.text)

  if len(contents)==10:
    # Add each element of content to the corresponding row
    for i, row in enumerate(rows):
        row['C'] = contents[i].strip()
    return 'success',rows
  else:
    rows.append({"content":contents})
    print("FAIL")
    return 'fail',rows


In [ ]:
print(get_muti_c(prompt_rows[0:5]))

FAIL
('fail', [{'Q': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'A': 'Natalia sold 48/2 = 24 clips in May. Natalia sold 48+24 = 72 clips altogether in April and May. The answer is 72', 'C': 'He sold [48/2 = y1] clips in May. Altogether he sold [48+y1 = y2]. The answer is y2.'}, {'Q': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'A': 'Weng earns 12/60 = $0.2 per minute. Working 50 minutes, she earned 0.2 x 50 = $10. The answer is 10', 'C': 'Weng earns [12/60 = y1] per minute. Working [50 = y2] minutes, she earned [0.2 * y2 = y3]. The answer is y3.'}, {'Q': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy 

In [ ]:
input_prompt=generate_continuation_gemini(prompt_rows[0:5])
print(input_prompt)

You are an agent that converts math problems to a Chain of Equations. Examples to generate Chain of Equations( C)  given Q and A are:

### EXAMPLES
Example 1
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees will the grove
workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21-15=6. The answer is 6.
C: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been [21 - 15 = y1]. The answer is y1.

Example 2
Q: The flowers cost $9, the clay pot costs $20 more than the flower, and the bag of soil costs $2 less than the flower. How much does it cost to plant the flowers?
A: The clay pot costs $20 + $9 = $29. The bag of soil costs $9 - $2 = $7. The cost to plant the flowers is $9 + $29 + $7 = $45. The answer is 45.
C: The clay pot costs [20 + 9 = y1]. The bag of soil

In [ ]:
len(prompt_rows)

7473

In [ ]:
prompt_rows=prompt_rows[:2000]

In [ ]:


generated_dataset = []
failed = []

for i in tqdm(range(0, len(prompt_rows), 10)):
    batch = prompt_rows[i:i+10]
    status, gen_batch = get_muti_c(batch)
    if status == 'success':
        generated_dataset.extend(gen_batch)
    else:
        failed.append(gen_batch)

    # Save generated_dataset to DataFrame pickle file after every 10 iterations
    if (i + 1) % 10 == 0:
        df = pd.DataFrame(generated_dataset)
        df.to_pickle(f'"/content/drive/MyDrive/NLP project/generated_dataset_{i+1}.pkl')
        print(f'Saved generated_dataset_{i+1}.pkl')

        if len(failed) > 0:
          with open('failed_data.pkl', 'wb') as file:
            pickle.dump(failed, file)
          print('Saved failed_data.pkl')

    time.sleep(4)



# Save any remaining data
if len(generated_dataset) > 0:
    df = pd.DataFrame(generated_dataset)
    df.to_pickle('/content/drive/MyDrive/NLP project/generated_dataset_final.pkl')
    print('Saved generated_dataset_final.pkl')

if len(failed) > 0:
    with open('failed_data.pkl', 'wb') as file:
      pickle.dump(failed, file)
    print('Saved failed_data.pkl')

 22%|██▎       | 45/200 [15:04<49:50, 19.30s/it]

FAIL


 24%|██▍       | 49/200 [16:22<48:41, 19.35s/it]

FAIL


 25%|██▌       | 50/200 [16:59<1:02:10, 24.87s/it]

...

FAIL


 91%|█████████ | 182/200 [59:52<07:01, 23.39s/it]

FAIL


100%|██████████| 200/200 [1:05:54<00:00, 19.77s/it]


Saved generated_dataset_final.pkl
Saved failed_data.pkl


In [ ]:
len(generated_dataset)

1900

In [ ]:
df[1790:1800]

,Q,A,C
1790,"Jane, Kyla, and Anthony have summer jobs in a ...",There are 1 x 60 minutes = 60 minutes in 1 hou...,There are [60/5 = y1] sets of 5 minutes in 1 h...
1791,"After sharing 100 stickers with her friends, X...",Five sheet have 5 x 10 = 50 stickers. Therefor...,There are [5 * 10 = y1] stickers on 5 sheets. ...
1792,Tamika drove for 8 hours at an average speed o...,Tamika drove 8 hours * 45 mph = 360 miles Loga...,Tamika drove [8 hours * 45 mph = y1] miles Log...
1793,James gets paid $0.50/mile to drive a truck ca...,First find the total payment James receives: $...,First find the total payment James receives: [...
1794,Charlotte is a dog walker and is creating a sc...,"On Monday, she spends 4 poodles * 2 hours = 8 ...","On Monday, she spends [4 poodles * 2 hours = y..."
1795,Natalia is riding a bicycle for the cycling co...,On Wednesday she covered half of the distance ...,On Wednesday she covered half of the distance ...
1796,"Yesterday, Bruce and Michael were playing foot...",Bruce scored 4 goals Scoring 3 times more than...,"Scoring 3 times more than Bruce, Michael score..."
1797,Mason is trying to download a 880 MB game to h...,First find how many MB are left to download: 8...,First find how many MB are left to download: [...
1798,Walter wants to serve hushpuppies to his guest...,He wants to serve 5 hushpuppies to each guest ...,He wants to serve [5 hushpuppies to each guest...
1799,"At the new store, there are 3 times as many gr...",Total number of candies is 200/2.5=80 candies....,Total number of candies is [200/2.5 = y1] cand...


In [ ]:
df.iloc[1790]['A']

'There are 1 x 60 minutes = 60 minutes in 1 hour. There are 60/5 = 12 sets of 5 minutes in 1 hour. So, Jane can fold 3 x 12 = 36 towels in an hour. There are 60/10 = 6 sets of 10 minutes in 1 hour. So, Kyla can fold 5 x 6 = 30 towels in an hour. There are 60/20 = 3 sets of 20 minutes in 1 hour. So, Anthony can fold 7 x 3 = 21 towels in an hour. Therefore, the 3 of them can fold a total of 36 + 30 + 21 = 87 towels in 1 hour. The answer is 87'

In [ ]:
df.iloc[1790]['C']

'There are [60/5 = y1] sets of 5 minutes in 1 hour. So, Jane can fold [3 * y1 = y2] towels in an hour. There are [60/10 = y3] sets of 10 minutes in 1 hour. So, Kyla can fold [5 * y3 = y4] towels in an hour. There are [60/20 = y5] sets of 20 minutes in 1 hour. So, Anthony can fold [7 * y5 = y6] towels in an hour. Therefore, the 3 of them can fold a total of [y2 + y4 + y6 = y7] towels in 1 hour. The answer is y7'

## CHECKING OUTPUTS

In [9]:
#df=pd.read_pickle('/content/drive/MyDrive/NLP project/generated_dataset_final.pkl')
df=pd.read_pickle('/content/drive/MyDrive/NLP project/testing data/gemma1_math_test.pkl')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
##sympy setup
!git clone https://github.com/sympy/sympy.git
import os
os.chdir('sympy/')
os.getcwdb()
!python -m pip install -e .

^C


FileNotFoundError: [Errno 2] No such file or directory: 'sympy/'

In [20]:
from sympy import *
import re

def equation_solver(text):

  matches = re.findall(r'\[(.*?)\]', text)
  answer=text.split("The answer is ")[-1].replace(".","")
  answer=symbols(answer)
  list_eq=[]
  for i in matches:
    eq=sympify("Eq(" + i.replace("=", ",") + ")")
    list_eq.append(eq)

  result =solve(list_eq)
  print(result)
  if(str(type(result))=="<class 'list'>"):
    result=result[0]

  if(answer in result.keys()):
    return int(result[answer])
  else:
    return list(result.values())

In [21]:
df.iloc[0]['Q']

'Harper decides to go to a nearby animal farm that has a total of 11 animals. He is standing in a queue near the entry gate, from the gate he could only see legs of the animals in the farm and he counted the total number of legs to be 30. He already knows that there are only ducks and horses in this farm. From this information find the number of ducks'

In [22]:
df.iloc[0]['A']

'7'

In [23]:
text=df.iloc[0]['sft_response']
print(text)

C: He could only see legs of ducks and horses in the farm, so the number of ducks is [30/4 = y1]. The number of horses is [30/4 = y2]. The answer is y1 and y2 both are horses.


In [24]:
matches = re.findall(r'\[(.*?)\]', text)
answer=text.split("The answer is ")[-1].replace(".","")
answer=symbols(answer)
print(matches)
print(answer)

['30/4 = y1', '30/4 = y2']
(y1, and, y2, both, are, horses)


In [25]:
list_eq=[]
for i in matches:
  eq=sympify("Eq(" + i.replace("=", ",") + ")")
  list_eq.append(eq)
list_eq

[Eq(15/2, y1), Eq(15/2, y2)]

In [26]:
solve(list_eq)

{y1: 15/2, y2: 15/2}

In [27]:
equation_solver(text)

{y1: 15/2, y2: 15/2}


[15/2, 15/2]

In [28]:
df.head(1)

,Q,A,C,test_prompt,sft_response
0,Harper decides to go to a nearby animal farm t...,7,First search [duck number of legs -Wiki-> y1]....,### INSTRUCTION\nYour task is to generate a ch...,C: He could only see legs of ducks and horses ...


In [16]:
solvable=[]
for i in tqdm(range(df.shape[0])):
  row=df.iloc[i].to_dict()
  text=row['sft_response']
  try:
    ans=equation_solver(text)
    row["solved_answer"]=ans
    solvable.append(row)
  except:
    ans='not solvable'
    print(ans,row)

 14%|█▎        | 27/200 [00:00<00:01, 143.67it/s]

not solvable {'Q': 'In a parking lot, There are 10 cars and 2 bikes. Find out the number of wheels in that parking lot.', 'A': '44', 'C': 'First search [car number of wheels -Wiki-> y1]. A car has [y1 -QA(How many wheels does a car have?)-> y2] wheels each. Then search [bike number of wheels -Wiki-> y3]. A bike has [y3 -QA(How many wheels does a bike have?)-> y4] wheels each. So, there are [10*y2 + 2*y4 = y5] wheels in total.', 'test_prompt': '### INSTRUCTION\nYour task is to generate a chain of abstractions (C) for the given question (Q) using the available tools: Wiki, QA, and Mathematical. You can use a single tool or a combination of tools to derive the answer (C). Follow the rules and formats provided for each tool:                               \n**Tools:** \n1. **Wiki Tool:** Retrieves relevant articles from Wikipedia. * **Format:** `[search query -Wiki-> search query output]` \n2. **QA Tool:** Extracts focused answers from Wikipedia articles. * **Format:** `[input context -QA(q

 21%|██        | 42/200 [00:00<00:01, 116.65it/s]

not solvable {'Q': 'Everyday Isaac wakes up at 7 in the morning and goes to sleep at 11 in the night. For how many hours does Isaac sleep in a day', 'A': '8', 'C': 'First search [number of hours in a day -Wiki-> y1]. A day has [y1 -QA(How many hours in a day?)-> y2] hours. Isaac wakes up at 7 in the morning and goes to sleep at 11 in the night. Hence, Isaac sleeps for [y2 - (11 - 7) = 8] hours in a day.', 'test_prompt': '### INSTRUCTION\nYour task is to generate a chain of abstractions (C) for the given question (Q) using the available tools: Wiki, QA, and Mathematical. You can use a single tool or a combination of tools to derive the answer (C). Follow the rules and formats provided for each tool:                               \n**Tools:** \n1. **Wiki Tool:** Retrieves relevant articles from Wikipedia. * **Format:** `[search query -Wiki-> search query output]` \n2. **QA Tool:** Extracts focused answers from Wikipedia articles. * **Format:** `[input context -QA(question)-> output]` \n3

 46%|████▌     | 91/200 [00:00<00:00, 139.06it/s]

not solvable {'Q': 'Serenity bought a color box which has pencils corresponding to every color in the rainbow. 2 of Serenitys friends also bought the same color box. How many pencils they have in total.', 'A': '21', 'C': 'First search [number of colors in a rainbow -Wiki-> y1]. A color box has pencils corresponding to every color in the rainbow. Hence, the total number of pencils they have is [(y1 + 1) * 3 = 21].', 'test_prompt': '### INSTRUCTION\nYour task is to generate a chain of abstractions (C) for the given question (Q) using the available tools: Wiki, QA, and Mathematical. You can use a single tool or a combination of tools to derive the answer (C). Follow the rules and formats provided for each tool:                               \n**Tools:** \n1. **Wiki Tool:** Retrieves relevant articles from Wikipedia. * **Format:** `[search query -Wiki-> search query output]` \n2. **QA Tool:** Extracts focused answers from Wikipedia articles. * **Format:** `[input context -QA(question)-> ou

 77%|███████▋  | 154/200 [00:00<00:00, 215.89it/s]

not solvable {'Q': 'An garage has 20 bike wheels. How many bikes he can assemble using 20 wheels?', 'A': '10', 'C': 'First search [number of wheels per bike -Wiki-> y1]. A bike typically has [y1 -QA(How many wheels does a bike have?) -> y2] wheels. The garage has 20 bike wheels, so it can assemble [20 / y2 = 10] bikes.', 'test_prompt': '### INSTRUCTION\nYour task is to generate a chain of abstractions (C) for the given question (Q) using the available tools: Wiki, QA, and Mathematical. You can use a single tool or a combination of tools to derive the answer (C). Follow the rules and formats provided for each tool:                               \n**Tools:** \n1. **Wiki Tool:** Retrieves relevant articles from Wikipedia. * **Format:** `[search query -Wiki-> search query output]` \n2. **QA Tool:** Extracts focused answers from Wikipedia articles. * **Format:** `[input context -QA(question)-> output]` \n3. **Mathematical Tool:** Solves mathematical computations based on information returne

100%|██████████| 200/200 [00:01<00:00, 181.48it/s]

{'Q': 'Elias bought a dozen bananas. He ate 1 of the bananas. How many are left now?', 'A': '11', 'C': 'First search [number of bananas in a dozen -Wiki-> y1]. A dozen has [y1 -QA(How many bananas in a dozen?) -> y2] bananas. Elias bought a dozen bananas, so he has [y2 = y3] bananas. He ate 1 banana, thus he has [y3-1 = y4] bananas left. The answer is y4.', 'test_prompt': '### INSTRUCTION\nYour task is to generate a chain of abstractions (C) for the given question (Q) using the available tools: Wiki, QA, and Mathematical. You can use a single tool or a combination of tools to derive the answer (C). Follow the rules and formats provided for each tool:                               \n**Tools:** \n1. **Wiki Tool:** Retrieves relevant articles from Wikipedia. * **Format:** `[search query -Wiki-> search query output]` \n2. **QA Tool:** Extracts focused answers from Wikipedia articles. * **Format:** `[input context -QA(question)-> output]` \n3. **Mathematical Tool:** Solves mathematical comp

In [29]:
len(solvable)

41

In [30]:
df2=pd.DataFrame(solvable)

In [ ]:
df2['final_answer'] = df2['A'].apply(lambda x: x.split("answer is ")[-1])
df2['final_answer'] = pd.to_numeric(df2['final_answer'])

In [21]:
df2.to_csv('testcheck.csv')

In [33]:
df2.head()

,Q,A,C,test_prompt,sft_response,solved_answer,is_final_in_solved
0,Harper decides to go to a nearby animal farm t...,7,First search [duck number of legs -Wiki-> y1]....,### INSTRUCTION\nYour task is to generate a ch...,C: He could only see legs of ducks and horses ...,"[15/2, 15/2]",False
1,"At the end of the month, Hudson was checking h...",15,First search [number of days in December -Wiki...,### INSTRUCTION\nYour task is to generate a ch...,C: [465 = y1]. Hudson spent y1 dollars on burg...,465,False
2,Everyday Charlotte spends 16 hours on his phon...,56,First search [number of hours in a week -Wiki-...,### INSTRUCTION\nYour task is to generate a ch...,C: He spends [16/2 = y1] hours on social media...,[8],False
3,Robert started preparation for GRE entrance ex...,May,First search [number of months in a year -Wiki...,### INSTRUCTION\nYour task is to generate a ch...,C: Robert started preparation for GRE entrance...,[8],False
4,Noah is supposed to visit a doctor 3 times eve...,36,First search [number of months in a year -Wiki...,### INSTRUCTION\nYour task is to generate a ch...,C: Noah visited a doctor 3 times every month f...,[36],True


In [24]:
def check_answer(row):
  row['final_answer']=str(row['answer'])
  row['solved_answer']=str(row['solved_answer'])
  if(str(row['solved_answer']).find("[")==-1):
    return row['final_answer']==row['solved_answer']
  else:
    #print(row['solved_answer'])
    return row['final_answer'] in row['solved_answer']

# Apply the function to each row
df2['is_final_in_solved'] = df2.apply(check_answer, axis=1)

# Count the rows where final_answer is present in solved_answer list
count_valid = df2[df2['is_final_in_solved']].shape[0]

In [31]:
def check_answer(row):
    gen = row['solved_answer']
    act = row['A']
    if str(gen).find(str(act)) != -1:
        return True
    if str(act).find(str(gen)) != -1:
        return True
    if str(gen) == str(act):
        return True

    return False

# Apply the function to each row
df2['is_final_in_solved'] = df2.apply(check_answer, axis=1)

# Count the rows where final_answer is present in solved_answer list
count_valid = df2[df2['is_final_in_solved']].shape[0]

In [ ]:
df2.to_csv('b.csv')

In [32]:
count_valid

9

In [29]:
df2.head()

,Q,answer,A,test_prompt,sft_response,solved_answer,is_final_in_solved
0,"In a truck, there are 26 pink hard hats, 15 gr...",If there were 26 pink hard hats and Carl took ...,43,### INSTRUCTION\nYour task is to generate a ch...,"C: In the truck, there are [26 + 15 + 24 = y1]...",50,False
1,Dale and Andrew had breakfast at a cafe. A sli...,The cost of Dale's toast is 2 × $1 = $2. The c...,15,### INSTRUCTION\nYour task is to generate a ch...,C: Dale had [2 * 1 = y1] slices of toast each ...,21,False
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $50...",5,### INSTRUCTION\nYour task is to generate a ch...,"C: Betty needs $100 for the wallet, but she on...","[y4 + 45, 55 - y4, 70 - y4, 100 - y4]",True
3,Jesse and Mia are competing in a week long rac...,Jesse runs 2 miles in the first three days bec...,6,### INSTRUCTION\nYour task is to generate a ch...,C: Jesse runs [2/3 * 3 = y1] miles in the firs...,2,False
4,A bear is preparing to hibernate for the winte...,The bear gained 1 / 5 * 1000 = 200 pounds from...,200,### INSTRUCTION\nYour task is to generate a ch...,C: The bear needs to gain [1000 = y1] pounds. ...,200,True


In [35]:
df2.to_csv("/content/drive/MyDrive/NLP project/testing data/gemma1_solved.pkl")

In [ ]:
df2.to_pickle('/content/drive/MyDrive/NLP project/Maths_solved_800.pkl')

In [ ]:
solved_rows_df = df2[df2['is_final_in_solved'] == True]
solved_rows_df.shape

(810, 6)

In [ ]:
selected_rows = solved_rows_df.sample(n=600, random_state=42)  # Adjust the sampling criteria as needed
selected_rows.to_pickle('/content/drive/MyDrive/NLP project/atFinal-maths600.pkl')

In [ ]:
selected_rows.head()

,Q,A,C,solved_answer,final_answer,is_final_in_solved
279,Bucky earns money each weekend catching and se...,He is $25 short for the game because 60 - 35 =...,He is [60 - 35 = y1] dollars short for the gam...,2,2,True
756,Two brick walls are being built. Each wall has...,Each brick wall needs 30 x 50 = 1500 bricks. T...,"[30 * 50 = y1], [y1 * 2 = y2]. The answer is y2.",3000,3000,True
259,"Every day, Sara bakes 10 cakes and puts them i...","After 5 days, Sara has baked 5 days * 10 cakes...","After 5 days, Sara has baked [5 * 10 = y1] cak...",76,76,True
325,3/5 of the mangoes on a mango tree are ripe. I...,The number of ripe mangoes from the tree is 3/...,The number of ripe mangoes from the tree is [3...,96,96,True
599,"Mrs. Finley's class has 24 students, and Mr. J...",Half the number in Mrs. Finley's class is 24/2...,Half the number in Mrs. Finley's class is [24/...,22,22,True


In [ ]:
#800 out of 1800 valid

### CHAT MODEL PROMPT

In [ ]:
Examples to generate Chain of Equations( C)  given Q and A are:

Example 1
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees will the grove
workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21-15=6. The answer is 6.
C: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been [21 - 15 = y1]. The answer is y1.

Example 2
Q: The flowers cost $9, the clay pot costs $20 more than the flower, and the bag of soil costs $2 less than the flower. How much does it cost to plant the flowers?
A: The clay pot costs $20 + $9 = $29. The bag of soil costs $9 - $2 = $7. The cost to plant the flowers is $9 + $29 + $7 = $45. The answer is 45.
C: The clay pot costs [20 + 9 = y1]. The bag of soil costs [9 - 2 = y2]. The cost to plant the flowers is [9 + y1 + y2 = y3]. The answer is y3.

Example 3
Q: From March to August, Sam made $460 doing 23 hours of yard work. However, from September to February, Sam was only able to work for 8 hours. If Sam
is saving up to buy a video game console that costs $600 and has already spent $340 to fix his car, how many more hours does he need to work before he can buy
the video game console?
A: Sam makes $460 / 23 hrs = $20/hr. From September to February, Sam made 8hrs x $20/hr = $160. From March to February, Sam made a total of $460 + $160
= $620. After fixing his car, he was left with $620 - $340 = $280. Sam needs another $600 - $280 = $320. Sam needs to work another $320 / $20/hr = 16 hours.
The answer is 16.
C: Sam makes [460 / 23 = y1] dollars per hour. From September to February, Sam made [8 * y1 = y2] dollars. From March to February, Sam made a
total of [460 + y2 = y3] dollars. After fixing his car, he was left with [y3 - 340 = y4]. Sam needs another [600 - y4 = y5] dollars. Sam needs to work
another [y5 / y1 = y6] hours. The answer is y6.

Example 4
Q: There were nine computers in the server room. Five more computers were installed each day, from Monday to Thursday. How many computers are now in
the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
C: There were originally 9 computers. For each of 4 days, 5 more computers were added. So [5 * 4 = y1] computers were added. [9 + y1 = y2].
The answer is y2.

Example 5
Q: Of the 90 people on William’s bus, 3/5 were Dutch. Of the 1/2 of the Dutch who were also American, 1/3 got window seats. What’s the number of Dutch
Americans who sat at the windows?
A: On the bus, the number of Dutch people was 3/5 of the total number, a total of 3/5 x 90 = 54 people. Out of the 54 people who were Dutch, 1/2 were Dutch
Americans, a total of 1/2 x 54 = 27 people. If 1/3 of the passengers on the bus identifying as Dutch Americans sat at the windows, their number is 1/3 x 27 = 9.
The answer is 9.
C: On the bus, the number of Dutch people was 3/5 of the total number, a total of [3/5 * 90 = y1] people. Out of the Dutch people, 1/2 were Dutch
Americans, a total of [1/2 * y1 = y2] people. If 1/3 of the passengers on the bus identifying as Dutch Americans sat at the windows, their number
is [1/3 * y2 = y3]. The answer is y3.

###

Now  generate C for the test examples below. Please Write "###DONE" when you have completed generating C for all of the examples below.

Test Example 1
Q: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
A: He writes each friend 3*2=6 pages a week So he writes 6*2=12 pages every week That means he writes 12*52=624 pages a year The answer is 624
C:

Test Example 2
Q: Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
A: There are 80/100 * 10 = 8 more purple flowers than yellow flowers. So in Mark's garden, there are 10 + 8 = 18 purple flowers. Purple and yellow flowers sum up to 10 + 18 = 28 flowers. That means in Mark's garden there are 25/100 * 28 = 7 green flowers. So in total Mark has 28 + 7 = 35 plants in his garden. The answer is 35
C:

Test Example 3
Q: Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?
A: He eats 32 from the largest pizzas because 2 x 16 = 32 He eats 16 from the small pizza because 2 x 8 = 16 He eats 48 pieces because 32 + 16 = 48 The answer is 48
C:

Test Example 4
Q: Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds?
A: To the initial 2 pounds of jelly beans, he added enough brownies to cause the weight to triple, bringing the weight to 2*3=6 pounds. Next, he added another 2 pounds of jelly beans, bringing the weight to 6+2=8 pounds. And finally, he added enough gummy worms to double the weight once again, to a final weight of 8*2=16 pounds. The answer is 16
C:

Test Example 5
Q: Alexis is applying for a new job and bought a new set of business clothes to wear to the interview. She went to a department store with a budget of $200 and spent $30 on a button-up shirt, $46 on suit pants, $38 on a suit coat, $11 on socks, and $18 on a belt. She also purchased a pair of shoes, but lost the receipt for them. She has $16 left from her budget. How much did Alexis pay for the shoes?
A: Let S be the amount Alexis paid for the shoes. She spent S + 30 + 46 + 38 + 11 + 18 = S + 143. She used all but $16 of her budget, so S + 143 = 200 - 16 = 184. Thus, Alexis paid S = 184 - 143 = $41 for the shoes. The answer is 41
C:

###Examples to generate Chain of Equations( C)  given Q and A are:

Example 1
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees will the grove
workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21-15=6. The answer is 6.
C: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been [21 - 15 = y1]. The answer is y1.

Example 2
Q: The flowers cost $9, the clay pot costs $20 more than the flower, and the bag of soil costs $2 less than the flower. How much does it cost to plant the flowers?
A: The clay pot costs $20 + $9 = $29. The bag of soil costs $9 - $2 = $7. The cost to plant the flowers is $9 + $29 + $7 = $45. The answer is 45.
C: The clay pot costs [20 + 9 = y1]. The bag of soil costs [9 - 2 = y2]. The cost to plant the flowers is [9 + y1 + y2 = y3]. The answer is y3.

Example 3
Q: From March to August, Sam made $460 doing 23 hours of yard work. However, from September to February, Sam was only able to work for 8 hours. If Sam
is saving up to buy a video game console that costs $600 and has already spent $340 to fix his car, how many more hours does he need to work before he can buy
the video game console?
A: Sam makes $460 / 23 hrs = $20/hr. From September to February, Sam made 8hrs x $20/hr = $160. From March to February, Sam made a total of $460 + $160
= $620. After fixing his car, he was left with $620 - $340 = $280. Sam needs another $600 - $280 = $320. Sam needs to work another $320 / $20/hr = 16 hours.
The answer is 16.
C: Sam makes [460 / 23 = y1] dollars per hour. From September to February, Sam made [8 * y1 = y2] dollars. From March to February, Sam made a
total of [460 + y2 = y3] dollars. After fixing his car, he was left with [y3 - 340 = y4]. Sam needs another [600 - y4 = y5] dollars. Sam needs to work
another [y5 / y1 = y6] hours. The answer is y6.

Example 4
Q: There were nine computers in the server room. Five more computers were installed each day, from Monday to Thursday. How many computers are now in
the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
C: There were originally 9 computers. For each of 4 days, 5 more computers were added. So [5 * 4 = y1] computers were added. [9 + y1 = y2].
The answer is y2.

Example 5
Q: Of the 90 people on William’s bus, 3/5 were Dutch. Of the 1/2 of the Dutch who were also American, 1/3 got window seats. What’s the number of Dutch
Americans who sat at the windows?
A: On the bus, the number of Dutch people was 3/5 of the total number, a total of 3/5 x 90 = 54 people. Out of the 54 people who were Dutch, 1/2 were Dutch
Americans, a total of 1/2 x 54 = 27 people. If 1/3 of the passengers on the bus identifying as Dutch Americans sat at the windows, their number is 1/3 x 27 = 9.
The answer is 9.
C: On the bus, the number of Dutch people was 3/5 of the total number, a total of [3/5 * 90 = y1] people. Out of the Dutch people, 1/2 were Dutch
Americans, a total of [1/2 * y1 = y2] people. If 1/3 of the passengers on the bus identifying as Dutch Americans sat at the windows, their number
is [1/3 * y2 = y3]. The answer is y3.

###

Now  generate C for the test examples below using Q and A. Please Write "###DONE" when you have completed generating C for all of the examples below.

Test Example 1
Q: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
A: He writes each friend 3*2=6 pages a week So he writes 6*2=12 pages every week That means he writes 12*52=624 pages a year The answer is 624
C:

Test Example 2
Q: Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
A: There are 80/100 * 10 = 8 more purple flowers than yellow flowers. So in Mark's garden, there are 10 + 8 = 18 purple flowers. Purple and yellow flowers sum up to 10 + 18 = 28 flowers. That means in Mark's garden there are 25/100 * 28 = 7 green flowers. So in total Mark has 28 + 7 = 35 plants in his garden. The answer is 35
C:

Test Example 3
Q: Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?
A: He eats 32 from the largest pizzas because 2 x 16 = 32 He eats 16 from the small pizza because 2 x 8 = 16 He eats 48 pieces because 32 + 16 = 48 The answer is 48
C:

Test Example 4
Q: Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds?
A: To the initial 2 pounds of jelly beans, he added enough brownies to cause the weight to triple, bringing the weight to 2*3=6 pounds. Next, he added another 2 pounds of jelly beans, bringing the weight to 6+2=8 pounds. And finally, he added enough gummy worms to double the weight once again, to a final weight of 8*2=16 pounds. The answer is 16
C:

Test Example 5
Q: Alexis is applying for a new job and bought a new set of business clothes to wear to the interview. She went to a department store with a budget of $200 and spent $30 on a button-up shirt, $46 on suit pants, $38 on a suit coat, $11 on socks, and $18 on a belt. She also purchased a pair of shoes, but lost the receipt for them. She has $16 left from her budget. How much did Alexis pay for the shoes?
A: Let S be the amount Alexis paid for the shoes. She spent S + 30 + 46 + 38 + 11 + 18 = S + 143. She used all but $16 of her budget, so S + 143 = 200 - 16 = 184. Thus, Alexis paid S = 184 - 143 = $41 for the shoes. The answer is 41
C:

###

Respond only with C for the test examples. Use this format >> Test Example <number> C : <generated C>
Chain of Equations C for the above test examples are:

